This notebook is to develop and test lib/color_compass_code.py

TO DO
-  fix coloring problem when dimZ-dimX>1

# color_compass.py

In [1]:
### lib/color_compass.py ###
import stim
import numpy as np
from pymatching import Matching
import numpy as np
from termcolor import colored

In [2]:
from lib.color_compass import Lattice2D, compass_to_surface

# Surface code as a compass code

In [3]:
dim = 4
dimX,dimZ = dim,dim
lat = compass_to_surface(dimX,dimZ,'X')
print(lat)
for stab in lat.getS():
    print()
    for i in range(dimX):
        print(stab[dimX*i:dimX*(i+1)])

000---001---002---003
 |  #  |  #  |  #  |
004---005---006---007
 |  #  |  #  |  #  |
008---009---010---011
 |  #  |  #  |  #  |
012---013---014---015


XX__
XX__
____
____

__XX
__XX
____
____

____
X___
X___
____

____
_XX_
_XX_
____

____
___X
___X
____

____
____
XX__
XX__

____
____
__XX
__XX

ZZ__
____
____
____

__ZZ
____
____
____

_ZZ_
_ZZ_
____
____

____
____
_ZZ_
_ZZ_

____
ZZ__
ZZ__
____

____
____
____
ZZ__

____
__ZZ
__ZZ
____

____
____
____
__ZZ


# (Issue) Failed coloring
-  Working cases:
    - Square lattice
    - Lattice with dimX=1 or dimZ=1
    - Lattice with dimX>dimZ
- Failed cases:
    - dimZ-dimX>1 then stop working

In [9]:
dimX,dimZ = np.random.randint(1,10,size=2)
print(f'dimX = {dimX}, dimZ = {dimZ}\n')
coloring = ([-1,1]*int(np.ceil(dimZ/2)))
coloring = coloring[:dimZ-1] + coloring[1:dimZ]
coloring *= int(np.ceil(dimX/2))
coloring = coloring[:(dimZ-1)*(dimX-1)]
lat = Lattice2D(dimX, dimZ)
lat.color_lattice(coloring)
print(lat)

dimX = 6, dimZ = 8

000---001---002---003---004---005---006---007
 |  #  |  #  |  #  |  #  |  #  |  #  |  #  |
008---009---010---011---012---013---014---015
 |  #  |  #  |  #  |  #  |  #  |  #  |  #  |
016---017---018---019---020---021---022---023
 |  #  |  #  |  #  |  #  |  #  |  #  |  #  |
024---025---026---027---028---029---030---031
 |  #  |  #  |  #  |  #  |  #  |  #  |  #  |
032---033---034---035---036---037---038---039
 |  #  |  #  |  #  |  #  |  #  |  #  |  #  |
040---041---042---043---044---045---046---047

